<a href="https://colab.research.google.com/github/GN-Yu/TSRL-project/blob/main/one_dimension_optmized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import psutil
import subprocess
import time

import random
import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

# Function to execute shell command and return output
def run_shell_command(cmd):
  return subprocess.run(cmd, stdout=subprocess.PIPE, shell=True).stdout.decode('utf-8').strip()

# Check CPU
cpu_info = run_shell_command("cat /proc/cpuinfo | grep 'model name' | uniq")
print("CPU:", cpu_info.split(":")[1].strip() if cpu_info else "Not available")

# Number of CPU Cores
num_cores = os.cpu_count()
print("Number of CPU cores:", num_cores)

# Total RAM
ram_info = psutil.virtual_memory()
total_ram = ram_info.total / (1024 ** 3)  # Convert bytes to GB
print(f"Total RAM : {total_ram} GB")

# Check GPU
gpu_info = !nvidia-smi --query-gpu=gpu_name,memory.total --format=csv
if gpu_info and len(gpu_info) > 1:
  gpu_name = gpu_info[1].split(',')[0]
  gpu_memory_mib = float(gpu_info[1].split()[2])  # Extract the memory in MiB
  gpu_memory_gb = gpu_memory_mib / 1024
  print(f"GPU: {gpu_name}, {gpu_memory_gb} GB")
else:
  print("GPU: Not available")

# TensorFlow info
print(f"TensorFlow version: {tf.__version__}")
print("TensorFlow GPU Availability:", tf.test.is_gpu_available())


from google.colab import drive

# mount drive
drive.mount('/content/drive')

# organize files
for dirname, _, filenames in os.walk('/content/drive/MyDrive/TSRL_project'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

CPU: Intel(R) Xeon(R) CPU @ 2.30GHz
Number of CPU cores: 2
Total RAM : 12.674781799316406 GB


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU: Tesla T4, 15.0 GB
TensorFlow version: 2.15.0
TensorFlow GPU Availability: True


In [ ]:
def SEED_everything(seed=2023):
  np.random.seed(seed)
  tf.random.set_seed(seed)
  random.seed(seed)

def sample_brownian_motion(num_trajectories, time_steps):
  """
  Function to sample Brownian motion for the specified number of trajectories and time steps.

  :param num_trajectories: The number of trajectories to simulate.
  :param time_steps: A 1D TensorFlow tensor of time steps.
  :return: A TensorFlow tensor of shape (num_trajectories, len(time_steps)-1) representing the Brownian increments.
  """
  # Calculate time intervals (delta_t) as differences between consecutive time steps
  # Ensure time_steps is a TensorFlow tensor
  time_steps = tf.convert_to_tensor(time_steps, dtype=tf.float32)
  delta_t = tf.experimental.numpy.diff(time_steps)

  # Sample Brownian increments ∆W for each time interval and each trajectory
  # Ensure delta_t is broadcasted correctly
  stddev = tf.sqrt(tf.expand_dims(delta_t, axis=0))
  brownian_increments = tf.random.normal([num_trajectories, len(delta_t)], mean=0.0, stddev=stddev)

  return brownian_increments

def sample_initial_states(num_samples):
  # Implement initial state sampling here
  pass

def sample_levy_jumps(num_trajectories, lambda_param, mu, sigma, time_horizon, max_jumps=1000):
  """
  Function to sample jumps from a Lévy process for the specified number of trajectories.
  Returns a 3D NumPy array with shape (num_trajectories, max_jumps, 2), where the last dimension
  contains arrival times and jump sizes respectively.

  :param num_trajectories: The number of trajectories to simulate.
  :param lambda_param: The intensity parameter λ of the Poisson process.
  :param mu: Mean of the normal distribution for jump sizes.
  :param sigma: Standard deviation of the normal distribution for jump sizes.
  :param time_horizon: The total time horizon T.
  :param max_jumps: Maximum number of jumps to consider for each trajectory.
  :return: A 3D NumPy array containing (arrival_times, jump_sizes) for each trajectory.
  """
  all_jumps = np.zeros((num_trajectories, max_jumps, 2))

  for i in range(num_trajectories):
    # Generate a sequence of exponential distributions with parameter λ
    exponential_samples = np.random.exponential(1/lambda_param, max_jumps)

    # Compute the cumulative sum to get the arrival times of the Poisson process
    arrival_times = np.cumsum(exponential_samples)
    valid_indices = arrival_times <= time_horizon
    arrival_times = arrival_times[valid_indices]

    # Sample from the normal distribution for each arrival time to get the jump sizes
    jump_sizes = np.random.normal(mu, sigma, arrival_times.size)

    all_jumps[i, :arrival_times.size, 0] = arrival_times
    all_jumps[i, :arrival_times.size, 1] = jump_sizes

  # Find the largest non-zero index across all trajectories
  largest_non_zero_index = max(np.max(np.where(all_jumps.any(axis=2))[1]), 0)

  # Truncate each trajectory at this index
  truncated_jumps = all_jumps[:, :largest_non_zero_index + 1, :]

  return truncated_jumps


def create_neural_network(input_dim, layer_width=25, output_dim=2):
  # Define the input layer
  inputs = tf.keras.Input(shape=(input_dim,))

  # First linear layer
  x = tf.keras.layers.Dense(layer_width, activation='tanh')(inputs)

  # Add five residual blocks
  for _ in range(5):
      y = tf.keras.layers.Dense(layer_width, activation='tanh')(x)
      y = tf.keras.layers.Dense(layer_width, activation='tanh')(y)
      x = tf.keras.layers.Add()([x, y])

  # Final linear layer
  outputs = tf.keras.layers.Dense(output_dim)(x)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  return model

In [ ]:
def G_function(x, z):
  """Define the function G(x, z)."""
  return x * tf.exp(z)

def normal_pdf(z, mu, sigma):
  """Normal probability density function."""
  pi = tf.constant(np.pi)
  return (1 / (tf.sqrt(2 * pi) * sigma)) * tf.exp(-((z - mu) ** 2) / (2 * sigma ** 2))

def integral_G_phi(x, mu, sigma, G_func, phi_func):
  """Calculate the integral of G_func(x, z) * phi_func(z) over R for each element in x."""
  z = tf.linspace(-200., 70., 2**15+1)
  dz = (z[-1] - z[0]) / (int(z.shape[0]) - 1)

  # Prepare z for broadcasting: reshape z to [1, 1, ..., 1, len(z)]
  z_reshaped = tf.reshape(z, [1] * (len(x.shape) - 1) + [-1])

  # Calculate G_func and phi_func for each combination of x and z
  y = G_func(tf.expand_dims(x, -1), z_reshaped) * phi_func(z_reshaped, mu, sigma)

  # Apply the Simpson's Method for integration along the last dimension
  integral_values = (y[..., 0] + y[..., -1] + 4 * tf.reduce_sum(y[..., 1:-1:2], axis=-1) + 2 * tf.reduce_sum(y[..., 2:-1:2], axis=-1)) * dz / 3

  return integral_values

In [ ]:
def update_X(brownian_motions, levy_jumps, X, time_snapshots, timestep):
  time_diff = time_snapshots[timestep + 1] - time_snapshots[timestep]

  # Mask for non-zero jumps that occur in the current time interval
  masks = tf.logical_and(
    (levy_jumps[:, :, 0] > time_snapshots[timestep]),
    (levy_jumps[:, :, 0] <= time_snapshots[timestep + 1])
  )
  masks_float = tf.cast(masks, dtype=tf.float32)

  # Vectorized computation of jump contributions for each trajectory
  X_current = X[:, timestep][:, tf.newaxis]  # Reshape X to for broadcasting
  jump_contributions = tf.reduce_sum(G_function(X_current, levy_jumps[:, :, 1]) * masks_float, axis=1)

  # Apply integral_G_phi element-wise
  integral_contributions = time_diff * lambda_poisson * integral_G_phi(X[:, timestep], mu, sigma, G_function, normal_pdf)

  # Calculate X for next time step
  X[:, timestep + 1].assign(X[:, timestep] + jump_contributions - integral_contributions)
  return X

def error_calc(model, brownian_motions, levy_jumps, X, NN_outputs, time_snapshots, timestep):
  num_trajectories = X.shape[0]
  max_jump_num = levy_jumps.shape[1]
  time_diff = time_snapshots[timestep + 1] - time_snapshots[timestep]

  # Mask for non-zero jumps that occur in the current time interval
  masks = tf.logical_and(
    (levy_jumps[:, :, 0] > time_snapshots[timestep]),
    (levy_jumps[:, :, 0] <= time_snapshots[timestep + 1])
  )
  masks_float = tf.cast(masks, dtype=tf.float32)

  # Apply the condition to filter relevant jumps
  relevant_jumps = tf.where(tf.expand_dims(masks, axis=-1), levy_jumps, tf.zeros_like(levy_jumps))

  X_current_repeat = tf.tile(tf.reshape(X[:,timestep], [-1, 1]), [1, max_jump_num])
  stacked_tensor1 = tf.stack([tf.fill([num_trajectories, max_jump_num], time_snapshots[timestep]), X_current_repeat], axis=-1)
  stacked_tensor2 = tf.stack([tf.fill([num_trajectories, max_jump_num], time_snapshots[timestep]), X_current_repeat + G_function(X_current_repeat, levy_jumps[:, :, 1])], axis=-1)
  N1_subtraction = tf.reshape(model(tf.reshape(stacked_tensor2, [-1, 2]))[:,0] - model(tf.reshape(stacked_tensor1, [-1, 2]))[:,0], [num_trajectories, max_jump_num])

  jump_contribution = tf.reduce_sum(N1_subtraction * masks_float, axis=1)

  term1 = jump_contribution - time_diff * NN_outputs[:, timestep, 1]

  TD_error = tf.reduce_mean(tf.square(term1 + NN_outputs[:, timestep, 0] - NN_outputs[:, timestep+1, 0]))

  constraint_error = tf.abs(tf.reduce_mean(term1))

  return TD_error, constraint_error

In [ ]:
# Set random seed for reproducibility
SEED = 2023
SEED_everything(SEED)

# Constants
initial_learning_rate = 5e-5
lambda_poisson = 0.3
mu = 0.4
sigma = 0.25
T = 1.0  # Total time
Y0_exact = 1.0

def g_function(x):
  return x


M = 1000  # Number of samples/trajectories
Iterations = 400  # Number of iterations

# unit tests
# M = 50  # Number of samples/trajectories
# Iterations = 5  # Number of iterations

# Generate equally spaced time snapshots from 0 to T
N = 50   # Number of time intervals
time_snapshots = tf.linspace(0.0, T, N+1)

# Initialize NN model
d = 1  # problem's dimensionality
input_dim = d + 1


with tf.device('/GPU:0'):

  model = create_neural_network(input_dim)

  # Optimizer
  optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)

  # Ensure the model is set to training mode
  model.trainable = True

  # Randomly generate terminal state (normal with mean of intial value)
  X_terminal = tf.Variable(tf.random.normal([M], mean = 1, stddev = 0.1), dtype=tf.float32)

  Y0_error = tf.Variable(tf.zeros([Iterations, N]), dtype=tf.float32)

  for iteration in range(Iterations):
    start_time = time.time()  # Start time of the iteration

    brownian_motions = sample_brownian_motion(M, time_snapshots)
    levy_jumps = tf.convert_to_tensor(sample_levy_jumps(M, lambda_poisson, mu, sigma, T, 1000), dtype=tf.float32)
    max_jump_num = levy_jumps.shape[1]

    # Initialize arrays for X and Y
    X = tf.Variable(tf.zeros((M, N+1)), dtype=tf.float32)
    # Y = tf.Variable(tf.zeros((M, N+1)), dtype=tf.float32)
    NN_outputs = tf.Variable(tf.zeros((M, N+1, 2)), dtype=tf.float32)

    # Set initial and terminal value for X
    X[:, 0].assign(tf.ones(M))
    X[:, N].assign(X_terminal)

    NN_outputs[:, 0, :].assign(model(tf.stack([tf.fill([M], time_snapshots[0]), X[:, 0]], axis=1)))

    # Iterate over time steps
    for timestep in range(N):
      with tf.GradientTape(persistent=True) as tape:

        update_X(brownian_motions, levy_jumps, X, time_snapshots, timestep)

        NN_outputs[:, timestep, :].assign(model(tf.stack([tf.fill([M], time_snapshots[timestep]), X[:, timestep]], axis=1)))  # less efficiency but could lead to better precision
        NN_outputs[:, timestep+1, :].assign(model(tf.stack([tf.fill([M], time_snapshots[timestep+1]), X[:, timestep+1]], axis=1)))

        # Loss 1 and 4: TD error, and N1 N2 constraint error
        TD_error, constraint_error = error_calc(model, brownian_motions, levy_jumps, X, NN_outputs, time_snapshots, timestep)

        if timestep == N-1:
          X_terminal.assign(X[:, N])

        terminal_input = tf.stack([tf.fill([M], T), X_terminal], axis=1)

        tape.watch(terminal_input)  # Ensure terminal_input is being watched
        tape.watch(X_terminal)  # Ensure X_terminal is being watched

        NN_outputs_terminal = model(terminal_input)

        N1_gradient_terminal = tf.cast(tape.gradient(NN_outputs_terminal[:, 0], terminal_input)[:, 1], tf.float32)

        g_X_terminal = tf.cast(g_function(X_terminal), tf.float32)
        g_gradient = tf.cast(tape.gradient(g_X_terminal, X_terminal), tf.float32)

        # Loss 2: Y terminal value error
        Y_terminal_error = tf.cast(tf.reduce_mean(tf.square(NN_outputs_terminal[:, 0] - g_X_terminal)) / N, tf.float32)

        # Loss 3: Y terminal value derivative error
        Y_teriminal_deriv_error = tf.cast(tf.reduce_mean(tf.square(N1_gradient_terminal - g_gradient)) / N, tf.float32)

        loss = TD_error + Y_terminal_error + Y_teriminal_deriv_error + constraint_error

        Y0_error[iteration, timestep].assign(tf.reduce_mean(tf.abs((model(tf.stack([tf.fill([M], time_snapshots[0]), X[:, 0]], axis=1))[:, 0] - tf.fill([M], Y0_exact)) / Y0_exact)))

      # Compute gradients of the loss with respect to the model parameters
      training_gradients = tape.gradient(loss, model.trainable_variables)

      # Update the model parameters using the gradients
      optimizer.apply_gradients(zip(training_gradients, model.trainable_variables))

      # Print progress
      if timestep == 0:
        print(
          f"Iteration {iteration} Time step {timestep}: loss = {loss.numpy()}, TD_error = {TD_error}, Y_terminal_error = {Y_terminal_error}, Y_teriminal_deriv_error = {Y_teriminal_deriv_error}, constraint_error = {constraint_error}, Y0 error = {Y0_error[iteration, timestep]}"
          )

    iteration_duration = time.time() - start_time  # Duration of the iteration
    print(f"Iteration {iteration + 1} completed in {iteration_duration:.2f} seconds")

Iteration 0 Time step 0: loss = 0.013706374913454056, TD_error = 8.074993820628151e-05, Y_terminal_error = 0.003631183411926031, Y_teriminal_deriv_error = 0.001996913691982627, constraint_error = 0.007997527718544006, Y0 error = 0.23220419883728027
Iteration 1 completed in 29.41 seconds
Iteration 1 Time step 0: loss = 0.014726096764206886, TD_error = 2.8512225981103256e-05, Y_terminal_error = 0.0038005602546036243, Y_teriminal_deriv_error = 0.002415967173874378, constraint_error = 0.008481057360768318, Y0 error = 0.18622422218322754
Iteration 2 completed in 8.18 seconds
Iteration 2 Time step 0: loss = 0.017714789137244225, TD_error = 1.606309524504468e-05, Y_terminal_error = 0.006589506287127733, Y_teriminal_deriv_error = 0.0008177392301149666, constraint_error = 0.010291480459272861, Y0 error = 0.496798038482666
Iteration 3 completed in 8.22 seconds
Iteration 3 Time step 0: loss = 0.004235826898366213, TD_error = 1.7019532606354915e-05, Y_terminal_error = 0.0033555454574525356, Y_teri

KeyboardInterrupt: ignored

Iteration 0 Time step 0: loss = 0.007462481968104839, Y0 error = 0.23220449686050415

Iteration 1 completed in 17.68 seconds

Iteration 1 Time step 0: loss = 0.006107668858021498, Y0 error = 0.02882683277130127

Iteration 2 completed in 6.99 seconds

Iteration 0 Time step 0: loss = 0.00771487969905138, Y0 error = 0.23220448195934296

Iteration 1 completed in 11.94 seconds

Iteration 1 Time step 0: loss = 0.004558033309876919, Y0 error = 0.09524476528167725

Iteration 2 completed in 6.66 seconds

In [ ]:
# tensors = [TD_error, Y_terminal_error, Y_teriminal_deriv_error, constraint_error]
# for i, tensor in enumerate(tensors):
#     print(f"Tensor {i}: dtype={tensor.dtype}")


In [ ]:
checkpoint = tf.train.Checkpoint(Y0_error=Y0_error)
checkpoint.save('/content/drive/MyDrive/TSRL_project/y0_error.ckpt')

model.save('/content/drive/MyDrive/TSRL_project/my_model')  # SavedModel format

# model.save('/content/drive/MyDrive/TSRL_project/my_model.h5')  # HDF5 format